# TBH Secure Agents - Features Demo

This notebook demonstrates the key features of the TBH Secure Agents framework, including:

1. **Basic Usage**: Creating experts, operations, and squads
2. **Security Profiles**: Using different security profiles (minimal, low, standard)
3. **Guardrails**: Using template variables and conditional formatting
4. **Result Destination**: Saving results to files in various formats

We'll use minimal security settings throughout this notebook to make it easier to understand and run.

## Setup

First, let's install the TBH Secure Agents package and set up our environment.

In [ ]:
# Install the package
!pip install tbh-secure-agents

In [ ]:
# Import the necessary modules
from tbh_secure_agents import Expert, Operation, Squad
import os
import json
from IPython.display import display, Markdown, HTML

# Create an output directory for saving results
os.makedirs('output', exist_ok=True)

### Set your API Key

You need to set your Google API key to use the Gemini model. You can get an API key from the [Google AI Studio](https://makersuite.google.com/app/apikey).

In [ ]:
# Set your API key here
API_KEY = "your_api_key_here"  # Replace with your actual API key

# Alternatively, you can set it as an environment variable
# os.environ["GOOGLE_API_KEY"] = "your_api_key_here"

## 1. Basic Usage

Let's start with the basics: creating experts, operations, and squads.

### Creating Experts

An expert is an autonomous entity designed to perform specific operations. Each expert has a specialty, objective, and background.

In [ ]:
# Create a content writer expert
content_writer = Expert(
    specialty="Content Writer",
    objective="Create engaging and informative content",
    background="You are an experienced content writer with expertise in creating clear, concise, and engaging content.",
    api_key=API_KEY,
    security_level="minimal"  # Using minimal security for simplicity
)

# Create a data analyst expert
data_analyst = Expert(
    specialty="Data Analyst",
    objective="Analyze data and provide insights",
    background="You are a skilled data analyst with experience in interpreting data and extracting meaningful insights.",
    api_key=API_KEY,
    security_level="minimal"  # Using minimal security for simplicity
)

### Creating Operations

An operation defines a unit of work to be performed by an expert. Each operation has instructions and an assigned expert.

In [ ]:
# Create an operation for the content writer
content_operation = Operation(
    instructions="Write a short blog post about the benefits of artificial intelligence in healthcare.",
    output_format="A well-structured blog post with a title, introduction, main points, and conclusion.",
    expert=content_writer
)

# Create an operation for the data analyst
analysis_operation = Operation(
    instructions="Analyze the following data and provide insights: Patient wait times decreased by 30% after implementing AI scheduling. Diagnostic accuracy improved by 15%. Treatment planning time reduced by 25%.",
    output_format="A concise analysis with key insights and recommendations.",
    expert=data_analyst
)

### Creating and Deploying a Squad

A squad manages a group of experts and orchestrates the execution of operations.

In [ ]:
# Create a squad with both experts and operations
healthcare_squad = Squad(
    experts=[content_writer, data_analyst],
    operations=[content_operation, analysis_operation],
    process="sequential",  # Operations run in sequence
    security_level="minimal"  # Using minimal security for simplicity
)

# Deploy the squad
result = healthcare_squad.deploy()

# Display the result
display(Markdown(result))

## 2. Security Profiles

The framework supports different security profiles: minimal, low, standard, high, and maximum. Let's see how they affect the behavior of experts and squads.

In [ ]:
# Create experts with different security profiles
minimal_expert = Expert(
    specialty="Content Writer",
    objective="Create engaging content",
    api_key=API_KEY,
    security_level="minimal"  # Minimal security checks
)

low_expert = Expert(
    specialty="Content Writer",
    objective="Create engaging content",
    api_key=API_KEY,
    security_level="low"  # Low security checks
)

standard_expert = Expert(
    specialty="Content Writer",
    objective="Create engaging content",
    api_key=API_KEY,
    security_level="standard"  # Standard security checks (default)
)

In [ ]:
# Create operations for each expert
minimal_operation = Operation(
    instructions="Write a short paragraph about artificial intelligence.",
    expert=minimal_expert
)

low_operation = Operation(
    instructions="Write a short paragraph about artificial intelligence.",
    expert=low_expert
)

standard_operation = Operation(
    instructions="Write a short paragraph about artificial intelligence.",
    expert=standard_expert
)

In [ ]:
# Execute the operations and compare the results
print("Minimal Security:")
minimal_result = minimal_operation.execute()
display(Markdown(minimal_result))

print("\nLow Security:")
low_result = low_operation.execute()
display(Markdown(low_result))

print("\nStandard Security:")
standard_result = standard_operation.execute()
display(Markdown(standard_result))

## 3. Guardrails

Guardrails provide a way to pass dynamic inputs to your Squad during deployment. These inputs can be used to guide the experts' responses, enforce constraints, and provide additional context.

### Basic Template Variables

You can use template variables in expert profiles and operation instructions.

In [ ]:
# Create an expert with template variables
template_expert = Expert(
    specialty="Content Writer specializing in {domain}",
    objective="Create {content_type} content for {audience}",
    background="You have experience writing {tone} content about {domain}.",
    api_key=API_KEY,
    security_level="minimal"
)

# Create an operation with template variables
template_operation = Operation(
    instructions="Write a {length} article about {topic} for {audience}. The tone should be {tone}.",
    output_format="A well-formatted {content_type}",
    expert=template_expert
)

# Create a squad
template_squad = Squad(
    experts=[template_expert],
    operations=[template_operation],
    process="sequential",
    security_level="minimal"
)

# Define guardrail inputs
guardrails = {
    "domain": "technology",
    "content_type": "blog post",
    "audience": "beginners",
    "tone": "friendly",
    "length": "short",
    "topic": "machine learning basics"
}

# Deploy with guardrails
template_result = template_squad.deploy(guardrails=guardrails)

# Display the result
display(Markdown(template_result))

### Conditional Formatting with Select Syntax

The select syntax allows for conditional content based on guardrail values.

In [ ]:
# Create an expert
conditional_expert = Expert(
    specialty="Content Writer",
    objective="Create engaging content",
    api_key=API_KEY,
    security_level="minimal"
)

# Create an operation with conditional formatting
conditional_operation = Operation(
    instructions="""Write a short article about {topic}.

{tone, select,
  formal:Use a professional, academic tone suitable for scholarly publications.|  
  conversational:Use a friendly, approachable tone as if speaking directly to the reader.|  
  technical:Use precise technical language appropriate for experts in the field.
}

{include_examples, select,
  true:Include practical examples to illustrate key points.|  
  false:Focus on theoretical concepts without specific examples.
}
""",
    expert=conditional_expert
)

# Create a squad
conditional_squad = Squad(
    experts=[conditional_expert],
    operations=[conditional_operation],
    process="sequential",
    security_level="minimal"
)

# Define guardrail inputs
conditional_guardrails = {
    "topic": "artificial intelligence",
    "tone": "conversational",
    "include_examples": "true"
}

# Deploy with guardrails
conditional_result = conditional_squad.deploy(guardrails=conditional_guardrails)

# Display the result
display(Markdown(conditional_result))

## 4. Result Destination

The result_destination parameter allows you to automatically save operation and squad results to files in various formats.

### Operation Result Destination

For individual operations, the result_destination parameter is a simple string specifying the file path where the result should be saved.

In [ ]:
# Create an expert
destination_expert = Expert(
    specialty="Content Writer",
    objective="Create engaging content",
    api_key=API_KEY,
    security_level="minimal"
)

# Create operations with different result_destination formats
txt_operation = Operation(
    instructions="Write a short paragraph about renewable energy in TXT format.",
    expert=destination_expert,
    result_destination="output/energy.txt"
)

md_operation = Operation(
    instructions="Write a short paragraph about renewable energy in MD format.",
    expert=destination_expert,
    result_destination="output/energy.md"
)

json_operation = Operation(
    instructions="Write a short paragraph about renewable energy in JSON format.",
    expert=destination_expert,
    result_destination="output/energy.json"
)

html_operation = Operation(
    instructions="Write a short paragraph about renewable energy in HTML format.",
    expert=destination_expert,
    result_destination="output/energy.html"
)

# Execute the operations
txt_result = txt_operation.execute()
md_result = md_operation.execute()
json_result = json_operation.execute()
html_result = html_operation.execute()

# Display the results
print("TXT Result:")
print(txt_result)

print("\nMD Result:")
display(Markdown(md_result))

print("\nJSON Result:")
print(json_result)

print("\nHTML Result:")
display(HTML(html_result))

### Squad Result Destination

For squads, the result_destination parameter is a dictionary that specifies the format and file path for the result.

In [ ]:
# Create a squad with result_destination
destination_squad = Squad(
    experts=[destination_expert],
    operations=[txt_operation, md_operation],
    process="sequential",
    security_level="minimal",
    result_destination={
        "format": "json",
        "file_path": "output/squad_result.json"
    }
)

# Deploy the squad
squad_result = destination_squad.deploy()

# Display the result
print("Squad Result:")
print(squad_result)

# Read and display the saved JSON file
with open("output/squad_result.json", "r") as f:
    saved_result = json.load(f)
    
print("\nSaved JSON Result:")
print(json.dumps(saved_result, indent=2))

## Combining Features

Now let's combine all these features in a single example.

In [ ]:
# Create an expert with template variables
combined_expert = Expert(
    specialty="{role} specializing in {domain}",
    objective="Create {content_type} content for {audience}",
    background="You have experience in {domain} and creating content for {audience}.",
    api_key=API_KEY,
    security_level="minimal"
)

# Create an operation with template variables and result_destination
combined_operation = Operation(
    instructions="""Write a {length} article about {topic} for {audience}.
    
{tone, select,
  formal:Use a professional, academic tone suitable for scholarly publications.|  
  conversational:Use a friendly, approachable tone as if speaking directly to the reader.|  
  technical:Use precise technical language appropriate for experts in the field.
}

{format, select,
  markdown:Format the content using Markdown syntax.|  
  plain:Use plain text without any special formatting.
}
""",
    expert=combined_expert,
    result_destination="output/combined_result.md"
)

# Create a squad with result_destination
combined_squad = Squad(
    experts=[combined_expert],
    operations=[combined_operation],
    process="sequential",
    security_level="minimal",
    result_destination={
        "format": "html",
        "file_path": "output/combined_squad_result.html"
    }
)

# Define guardrail inputs
combined_guardrails = {
    "role": "Content Writer",
    "domain": "artificial intelligence",
    "content_type": "blog post",
    "audience": "beginners",
    "length": "short",
    "topic": "machine learning applications",
    "tone": "conversational",
    "format": "markdown"
}

# Deploy with guardrails
combined_result = combined_squad.deploy(guardrails=combined_guardrails)

# Display the result
display(Markdown(combined_result))

# Read and display the saved HTML file
with open("output/combined_squad_result.html", "r") as f:
    html_content = f.read()
    
print("\nSaved HTML Result (preview):")
print(html_content[:500] + "...")

## Conclusion

In this notebook, we've demonstrated the key features of the TBH Secure Agents framework:

1. **Basic Usage**: Creating experts, operations, and squads
2. **Security Profiles**: Using different security profiles (minimal, low, standard)
3. **Guardrails**: Using template variables and conditional formatting
4. **Result Destination**: Saving results to files in various formats

These features can be combined in various ways to create powerful multi-agent systems for a wide range of applications.